In [1]:
import numpy as np
import matplotlib.pyplot as plt
import osqp
from scipy import sparse
%matplotlib widget

np.set_printoptions(suppress=True,linewidth = 150)

In [2]:
# piecewise jerk path test

1
class Factor(object):
    def __init__(self):
        #set start / end state
        self.start_l = 0.0
        self.start_dl = 0.0
        self.start_ddl = 0.0
        self.end_l = -0.5
        self.end_dl = 0.0
        self.end_ddl = 0.0
        # 两点之间的间隔
        self.val = 0.5
        # 终点位置的S，起点认为是0.0
        self.end_s = 30.0
        # 点的个数
        self.n = int(self.end_s/self.val) + 1
        self.s_arr = np.linspace(0, self.end_s, self.n, endpoint=True)


        #设置bound
        #bound[0,:]表示上（左）边界
        #bound[1,:]表示下（右）边界
        #bound[2,:]表示（上+下）/2 = 中心线
        n = self.n
        val = self.val
        self.bound = np.zeros((3,n))
        self.bound[0,:] = 2.0
        self.bound[1,:] = -2.0
        self.bound[1 ,0:int(n/3)] = -1.0
        self.bound[1 ,int(n/3):int(2*n/3)] = 1.0
        self.bound[0 ,int(2*n/3):n] = 1.0
        self.bound[2,:] = (self.bound[0,:] + self.bound[1,:])/2

        #设置Dl和ddl的bound
        #在frenet坐标系下，dl~theta , ddl ~ kappa 这里采用代码中给的参数
        self.dl_bound = np.zeros((2,n))
        self.dl_bound[0,:] = 0.5
        self.dl_bound[1,:] = -0.5
        self.ddl_bound = np.zeros((2,n))
        self.ddl_bound[0,:] = 0.2
        self.ddl_bound[1,:] = -0.2
        self.dddl_bound = np.zeros((2,n))
        self.dddl_bound[0,:] = 0.076689453212659936
        self.dddl_bound[1,:] = -0.076689453212659936


        #weight 各阶的权重
        self.weight_x = 1
        self.weight_x_ref = np.ones(n)
        self.weight_dx = 100
        self.weight_ddx = 1000
        self.weight_dddx = 50000
        self.weight_end_x = 50
        self.weight_end_dx = 0
        self.weight_end_ddx = 0
        self.scale_factor_0 = 1
        self.scale_factor_1 = 10
        self.scale_factor_2 = 100
        self.delta_s_square = val * val



In [3]:
def solve(factor):
    # set factor
    n = factor.n
    val = factor.val
    start_l = factor.start_l
    start_dl = factor.start_dl
    start_ddl = factor.start_ddl
    end_l = factor.end_l
    end_dl = factor.end_dl
    end_ddl = factor.end_ddl
    weight_x = factor.weight_x  
    bound = factor.bound
    dl_bound = factor.dl_bound
    ddl_bound = factor.ddl_bound
    dddl_bound = factor.dddl_bound
    weight_x_ref = factor.weight_x_ref
    weight_dx = factor.weight_dx
    weight_ddx = factor.weight_ddx
    weight_dddx = factor.weight_dddx
    weight_end_x = factor.weight_end_x
    weight_end_dx = factor.weight_end_dx
    weight_end_ddx = factor.weight_end_ddx
    scale_factor_0 = factor.scale_factor_0
    scale_factor_1 = factor.scale_factor_1
    scale_factor_2 = factor.scale_factor_2
    delta_s_square = factor.delta_s_square
    
    #calculate kernel
    P = np.zeros((3*n,3*n))
    for i in range(0,n-1):
        P[i,i]= (weight_x + weight_x_ref[i])/(scale_factor_0^2)
    P[n-1, n-1] = (weight_x + weight_x_ref[i] + weight_end_x)/(scale_factor_0^2)

    for i in range(0,n-1):
        P[n+i,n+i]= weight_dx/(scale_factor_1^2)
    P[2*n-1, 2*n-1] = (weight_dx + weight_end_dx)/(scale_factor_1^2)

    P[2*n,2*n] = (weight_ddx+weight_dddx / delta_s_square) / (scale_factor_2^2)
    for i in range(1,n-1):
        P[2*n+i,2*n+i] = (weight_ddx+ 2*weight_dddx / delta_s_square) / (scale_factor_2^2)
    P[3*n-1,3*n-1] = (weight_ddx+weight_dddx / delta_s_square) +weight_end_ddx/ (scale_factor_2^2)

    for i in range(0, n-1):
        P[2*n+i+1, 2*n+i] = (-2.0 * weight_dddx / delta_s_square) / (scale_factor_2^2)



    #calculate offset
    q = np.zeros(3*n)
    for i in range(0,n-1):
        q[i] = -2.0*weight_x_ref[i]*bound[2,:][i] / scale_factor_0
    q[n-1] = -2.0*(weight_x_ref[n-1]*bound[2,:][n-1]+weight_end_x*end_l) / scale_factor_0
    q[2*n-1] = -2.0*(weight_end_dx*end_dl) / scale_factor_0
    q[3*n-1] = -2.0*(weight_end_ddx*end_ddl) / scale_factor_0


    #calculate affine constraint

    ###lb & ub
    lb = np.zeros(3*n)
    ub = np.zeros(3*n)
    A = np.zeros((3*n,3*n))
    lb[0:n] = bound[1,:]
    ub[0:n] = bound[0,:]
    lb[n:2*n] = dl_bound[1,:]
    ub[n:2*n] = dl_bound[0,:]
    lb[2*n:3*n] = ddl_bound[1,:]
    ub[2*n:3*n] = ddl_bound[0,:]
    for i in range(3*n):
        A[i,i] = 1

    ###math
    lb.resize(4*n-1)
    ub.resize(4*n-1)
    A.resize((4*n-1,3*n))
    for i in range(0 , n - 1):
        A[3*n+i,2*n+i] = -1.0
        A[3*n+i,2*n+i+1] = 1.0
        lb[3*n+i] = dddl_bound[1,:][i]*val
        ub[3*n+i] = dddl_bound[0,:][i]*val

    lb.resize(5*n-2)
    ub.resize(5*n-2)
    A.resize((5*n-2,3*n))
    for i in range(0 , n-1):
        A[4*n-1+i,n+i] = -1.0
        A[4*n-1+i,n+i+1] = 1.0
        A[4*n-1+i,2*n+i] = -0.5*val
        A[4*n-1+i,2*n+i+1] = -0.5*val
        lb[4*n-1+i] = 0.0
        ub[4*n-1+i] = 0.0

    lb.resize(6*n-3)
    ub.resize(6*n-3)
    A.resize((6*n-3,3*n))
    for i in range(0 , n-1):
        A[5*n-2+i,i] = -1.0
        A[5*n-2+i,i+1] = 1.0
        A[5*n-2+i,n+i] = -val
        A[5*n-2+i,2*n+i] = -delta_s_square / 3.0
        A[5*n-2+i,2*n+i+1] = -delta_s_square / 6.0
        lb[5*n-2+i] = 0.0
        ub[5*n-2+i] = 0.0

    lb.resize(6*n)
    ub.resize(6*n)
    A.resize((6*n,3*n))

    A[6*n-3,0] = 1.0
    lb[6*n-3] = start_l
    ub[6*n-3] = start_l

    A[6*n-2,0] = 1.0
    lb[6*n-2] = start_dl
    ub[6*n-2] = start_dl

    A[6*n-1,0] = 1.0
    lb[6*n-1] = start_ddl
    ub[6*n-1] = start_ddl


    printFlag = False
    if printFlag:
        print("upper bound:",bound[0,:],"\nlower bound:",bound[1,:],"\nx_ref:",bound[2,:])
        print("P:\n",np.around(P, decimals=2)) 
        print("q:\n",np.around(q, decimals=2))
        print("ub:\n",np.around(ub, decimals=2))
        print("lb:\n",np.around(lb, decimals=2))
        print("A:\n",np.around(A, decimals=2))
    P = 2*P
    P = sparse.csc_matrix(P)
    A = sparse.csc_matrix(A)

    #solve
    prob1 = osqp.OSQP()
    prob1.setup(P,q,A,lb,ub)
    res = prob1.solve()
    if printFlag:
        print(res.x)
    return res.x


In [4]:
factor = Factor()
opt_xy = solve(factor)

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 183, constraints m = 366
          nnz(P) + nnz(A) = 1029
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -7.2544e+01   6.01e-01   1.52e+01   1.00e-01   5.61e-04s
 150  -2.9037e+01   1.15e-04   2.37e-02   3.39e+00   1.90e-03s

status:               solved
number of iter

In [5]:
#plot
fig1 = plt.figure(dpi = 100 , figsize=(12, 8))
ax1 = fig1.add_subplot(3,1,1)
ax2 = fig1.add_subplot(3,1,2)
ax3 = fig1.add_subplot(3,1,3)

def plotRes(factor,opt_xy):
    s_arr = factor.s_arr
    bound = factor.bound
    dl_bound = factor.dl_bound
    ddl_bound = factor.ddl_bound
    end_l = factor.end_l
    n = factor.n
    ax1.clear()
    ax1.plot(s_arr,opt_xy[:n], ".--",label="s-l")
    ax1.plot(s_arr,bound[1,:], color="red",label="s-bound")
    ax1.plot(s_arr,bound[0,:], color="red")
    ax1.plot(s_arr,bound[2,:],".--", color="grey",label="s-ref")
    ax1.plot(s_arr[n-1],end_l, "o" ,color="black",)
    ax1.text(s_arr[n-1]-0.5,end_l+0.5,  'end_l = %.2f'%(end_l), fontsize=12)
    ax1.legend()

    ax2.clear()
    ax2.plot(s_arr,opt_xy[n:2*n] , ".--",label="s-dl")
    ax2.plot(s_arr,dl_bound[1,:], color="red",label="s-bound")
    ax2.plot(s_arr,dl_bound[0,:], color="red")
    ax2.legend()

    ax3.clear()
    ax3.plot(s_arr,opt_xy[2*n:3*n] , ".--",label="s-ddl")
    ax3.plot(s_arr,ddl_bound[1,:], color="red")
    ax3.plot(s_arr,ddl_bound[0,:], color="red")
    ax3.legend()
    
plotRes(factor,opt_xy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
from ipywidgets import AppLayout, FloatSlider, VBox

class Plotter:
    def __init__(self):
        self.facotr= Factor()
    def update_endl(self,change):
        self.facotr.end_l = change.new
        opt_xy = solve(self.facotr)
        plotRes(self.facotr,opt_xy)
    def update_weightx(self,change):
        self.facotr.weight_x = change.new
        opt_xy = solve(self.facotr)
        plotRes(self.facotr,opt_xy)
    def update_weightdx(self,change):
        self.facotr.weight_dx = change.new
        opt_xy = solve(self.facotr)
        plotRes(self.facotr,opt_xy)
    def update_weightendx(self,change):
        self.facotr.weight_end_x = change.new
        opt_xy = solve(self.facotr)
        plotRes(self.facotr,opt_xy)
    def update_weightrefx(self,change):
        self.facotr.weight_x_ref = change.new * np.ones(self.facotr.n)
        opt_xy = solve(self.facotr)
        plotRes(self.facotr,opt_xy)
        
plotter = Plotter()


endl_slider = FloatSlider(
    orientation='horizontal',
    description='end_l:',
    value=plotter.facotr.end_l,
    min=-2.0,
    max=2.0
)
endl_slider.layout.margin = '0px 30% 0px 30%'
endl_slider.layout.width = '40%' 
endl_slider.observe(plotter.update_endl, names='value')

weightx_slider = FloatSlider(
    orientation='horizontal',
    description='weight_x:',
    value=plotter.facotr.weight_x,
    step=1.0,
    min=0.0,
    max=100.0
)
weightx_slider.layout.margin = '0px 30% 0px 30%'
weightx_slider.layout.width = '40%' 
weightx_slider.observe(plotter.update_weightx, names='value')

weightdx_slider = FloatSlider(
    orientation='horizontal',
    description='weight_dx:',
    value=plotter.facotr.weight_dx,
    step=10.0,
    min=0.0,
    max=10000.0
)
weightdx_slider.layout.margin = '0px 30% 0px 30%'
weightdx_slider.layout.width = '40%' 
weightdx_slider.observe(plotter.update_weightdx, names='value')

weightendx_slider = FloatSlider(
    orientation='horizontal',
    description='w_end_x:',
    value=plotter.facotr.weight_end_x,
    step=5.0,
    min=0.0,
    max=100.0
)
weightendx_slider.layout.margin = '0px 30% 0px 30%'
weightendx_slider.layout.width = '40%' 
weightendx_slider.observe(plotter.update_weightendx, names='value')

weightrefx_slider = FloatSlider(
    orientation='horizontal',
    description='w_ref_x:',
    value=plotter.facotr.weight_x_ref[0],
    step=0.1,
    min=0.0,
    max=10.0
)
weightrefx_slider.layout.margin = '0px 30% 0px 30%'
weightrefx_slider.layout.width = '40%' 
weightrefx_slider.observe(plotter.update_weightrefx, names='value')


AppLayout(
    center=fig1.canvas,
    footer=VBox([weightrefx_slider,weightendx_slider,endl_slider,weightx_slider,weightdx_slider]),
    pane_heights=[0, 6, 1]
)

AppLayout(children=(VBox(children=(FloatSlider(value=1.0, description='w_ref_x:', layout=Layout(margin='0px 30…